# Distributed Data Classification with NeMo Curator's `MultilingualDomainClassifier`

This notebook demonstrates the use of NeMo Curator's `MultilingualDomainClassifier`. The [multilingual domain classifier](https://huggingface.co/nvidia/multilingual-domain-classifier) is used to classify the domain of texts in any of 52 languages, including English. It helps with data annotation, which is useful in data blending for foundation model training. Please refer to the NemoCurator Multilingual Domain Classifier Hugging Face page for more information about the multilingual domain classifier, including its output labels, here: https://huggingface.co/nvidia/multilingual-domain-classifier.

This tutorial requires at least 1 NVIDIA GPU with:
  - Volta™ or higher (compute capability 7.0+)
  - CUDA 12.x

Before running this notebook, see this [Installation Guide](https://docs.nvidia.com/nemo/curator/latest/admin/installation.html#admin-installation) page for instructions on how to install NeMo Curator. Be sure to use an installation method which includes GPU dependencies.

For more information about the classifiers, refer to our [Distributed Data Classification](https://docs.nvidia.com/nemo/curator/latest/curate-text/process-data/quality-assessment/distributed-classifier.html) documentation page.

In [ ]:
# Silence Curator logs via Loguru
import os

os.environ["LOGURU_LEVEL"] = "ERROR"

The following imports are required for this tutorial:

In [ ]:
import pandas as pd

from nemo_curator.core.client import RayClient
from nemo_curator.pipeline import Pipeline
from nemo_curator.stages.text.classifiers import MultilingualDomainClassifier
from nemo_curator.stages.text.io.reader.jsonl import JsonlReader
from nemo_curator.stages.text.io.writer.jsonl import JsonlWriter

To run a pipeline in NeMo Curator, we must start a Ray cluster. This can be done manually (see the [Ray documentation](https://docs.ray.io/en/latest/ray-core/starting-ray.html)) or with Curator's `RayClient`:

In [3]:
try:
    ray_client = RayClient()
    ray_client.start()
except Exception as e:
    msg = f"Error initializing Ray client: {e}"
    raise RuntimeError(msg) from e

# Initialize Read, Classification, and Write Stages

Functions in NeMo Curator are called stages. For this tutorial, we will initialize 3 stages: a JSONL file reader, a multilingual domain classification stage, and a JSONL file writer.

For this tutorial, we will create a sample JSONL file to use:

In [4]:
input_file_path = "./input_data_dir"

# Create sample dataset for the tutorial
text = [
    # Chinese
    "量子计算将彻底改变密码学领域。",
    # Spanish
    "Invertir en fondos indexados es una estrategia popular para el crecimiento financiero a largo plazo.",
    # English
    "Recent advancements in gene therapy offer new hope for treating genetic disorders.",
    # Hindi
    "ऑनलाइन शिक्षण प्लेटफार्मों ने छात्रों के शैक्षिक संसाधनों तक पहुंचने के तरीके को बदल दिया है।",
    # Bengali
    "অফ-সিজনে ইউরোপ ভ্রমণ করা আরও বাজেট-বান্ধব বিকল্প হতে পারে।",
    # Portuguese
    "Os regimes de treinamento para atletas se tornaram mais sofisticados com o uso de análise de dados.",
    # Russian
    "Стриминговые сервисы меняют способ потребления людьми телевизионного и киноконтента.",
    # Japanese
    "植物ベースの食生活を採用する人が増えるにつれて、ビーガンレシピの人気が高まっています。",
    # Vietnamese
    "Nghiên cứu về biến đổi khí hậu có vai trò quan trọng trong việc phát triển các chính sách môi trường bền vững.",
    # Marathi
    "टेलीमेडिसिन त्याच्या सोयी आणि सुलभतेमुळे अधिक लोकप्रिय झाले आहे.",
]
df = pd.DataFrame({"text": text})

try:
    os.makedirs(input_file_path, exist_ok=True)
    df.to_json(input_file_path + "/data.jsonl", orient="records", lines=True)
except Exception as e:
    msg = f"Error creating input file: {e}"
    raise RuntimeError(msg) from e

We can define the reader stage with:

In [5]:
# Read existing directory of JSONL files
read_stage = JsonlReader(input_file_path, files_per_partition=1)

The classifier stage is broken down under the hood into a tokenizer stage and a model inference stage. Tokenization is run on the CPU while model inference is run on the GPU. This means that behind the scenes, the `MultilingualDomainClassifier` stage is actually being broken down into 2 stages (some parameters and details omitted to avoid complexity, please refer to the documentation for more details):

```python
class TokenizerStage:
    self._resources = Resources(cpus=1)
    self.model_identifier = "nvidia/multilingual-domain-classifier"
    self.text_field = "text"
    self.padding_side = "right"
    ...
class ModelStage:
    self._resources = Resources(cpus=1, gpus=1)
    self.model_identifier = "nvidia/multilingual-domain-classifier"
    self.model_inference_batch_size = 256
    ...
```

Optionally, the classifier predictions may be filtered to include only texts with values listed in `filter_by`. If the `filter_by` parameter is set, then a third stage is added:

```python
def filter_by_category(self, value: str) -> bool:
    return value in self.filter_by

...

if self.filter_by is not None and len(self.filter_by) > 0:
    self.stages.append(Filter(filter_fn=self.filter_by_category, filter_field=...))
```

See the [API Reference](https://docs.nvidia.com/nemo/curator/latest/apidocs/stages/stages.text.classifiers.domain.html#stages.text.classifiers.domain.MultilingualDomainClassifier) for more information about the `MultilingualDomainClassifier` class.

In [ ]:
# Initialize the multilingual domain classifier
classifier_stage = MultilingualDomainClassifier()

# If desired, you may filter your dataset with:
# classifier_stage = MultilingualDomainClassifier(filter_by=["Science", "Health"])  # noqa: ERA001
# See full list of domains here: https://huggingface.co/nvidia/multilingual-domain-classifier

Finally, we can define a stage for writing the results:

In [ ]:
# Write results to a directory
output_file_path = "./multilingual_domain_classifier_results"

# Use mode="overwrite" to overwrite the output directory if it already exists
# This helps to ensure that the correct output is written
write_stage = JsonlWriter(output_file_path, mode="overwrite")

# Initialize Pipeline

In NeMo Curator, we use pipelines to run distributed data workflows using Ray. Pipelines take care of resource allocation and autoscaling to achieve enhanced performance and minimize GPU idleness.

For the distributed data classifiers, we are able to achieve speedups by ensuring that model inference is run in parallel across all available GPUs, while other stages such as I/O, tokenization, and filtering are run across all available CPUs. This is possible because Curator pipelines are composable, which allows each stage in a pipeline to run independently and with its own specified hardware resources.

In [8]:
classifier_pipeline = Pipeline(name="classifier_pipeline", description="Run a classifier pipeline")

# Add stages to the pipeline
classifier_pipeline.add_stage(read_stage)
classifier_pipeline.add_stage(classifier_stage)
classifier_pipeline.add_stage(write_stage)

Pipeline(name='classifier_pipeline', stages=[jsonl_reader(JsonlReader), multilingual_domain_classifier_classifier(MultilingualDomainClassifier), jsonl_writer(JsonlWriter)])

Composability is also what allows a classifier to sit between pre-processing and post-processing stages. Typical text pre-processing add-ons include text normalization (lowercasing, URL/email removal, Unicode cleanup) and language identification and filtering (to keep only target languages). A full pipeline may look something like:

```python
pipeline = Pipeline(name="full_pipeline")
pipeline.add_stage(read_stage)                # reader (JSONL/S3/etc.)
pipeline.add_stage(lang_id_stage)             # optional: language filter
pipeline.add_stage(classifier_stage)          # classifier
pipeline.add_stage(write_stage)               # writer (JSONL/Parquet)
```

# Run the  Classifier

Let's run the full classifier pipeline:

In [ ]:
# Run the pipeline
result = classifier_pipeline.run()

Since the pipeline ran to completion and the result was written to a JSONL file, we can shut down the Ray cluster with:

In [ ]:
try:
    ray_client.stop()
except Exception as e:  # noqa: BLE001
    print(f"Error stopping Ray client: {e}")

# Inspect the Output

The write stage returns a list of written files. We can read the output file as a Pandas DataFrame for inspection.

In [ ]:
# For simplicity, we take the first written file from the writer stage
# In real pipelines, the writer may return multiple files (shards) or objects
result_file = result[0].data[0]

result_df = pd.read_json(result_file, lines=True)
result_df.head()

,text,multilingual_domain_pred
0,量子计算将彻底改变密码学领域。,Science
1,Invertir en fondos indexados es una estrategia...,Finance
2,Recent advancements in gene therapy offer new ...,Health
3,ऑनलाइन शिक्षण प्लेटफार्मों ने छात्रों के शैक्ष...,Jobs_and_Education
4,অফ-সিজনে ইউরোপ ভ্রমণ করা আরও বাজেট-বান্ধব বিকল...,Travel_and_Transportation


We can see that the predictions were generated as expected.